In [ ]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
import numpy as np
import matplotlib.pyplot as plt

# データの読み込み
train_data_nn = pd.read_csv('selected_train_set.csv')
test_data_nn = pd.read_csv('selected_test_set.csv')

# 特徴量と目的変数の分離
X_train_nn = train_data_nn.drop(columns=['PassengerId','Survived'])
y_train_nn = train_data_nn['Survived']
X_test_nn = test_data_nn.drop(columns=['PassengerId','Survived'])
y_test_nn = test_data_nn['Survived']

# データの標準化
scaler_nn = StandardScaler()
X_train_scaled_nn = scaler_nn.fit_transform(X_train_nn)
X_test_scaled_nn = scaler_nn.transform(X_test_nn)

# 標準化されたデータをデータフレームに戻し、元のカラム名を保持
X_train_scaled_df_nn = pd.DataFrame(X_train_scaled_nn, columns=X_train_nn.columns, index=X_train_nn.index)
X_test_scaled_df_nn = pd.DataFrame(X_test_scaled_nn, columns=X_test_nn.columns, index=X_test_nn.index)
# ニューラルネットワークのパラメータグリッドの設定
param_grid_nn = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],  # 隠れ層の構成
    'activation': ['relu', 'tanh'],                   # 活性化関数
    'learning_rate_init': [0.001, 0.01],              # 学習率
    'max_iter': [200, 500]                            # 最大エポック数
}

# 重要度を格納するリスト
importances_list_nn = []
# 予測確率を格納するリスト
predicted_probabilities_nn = []
# 精度を格納するリスト
accuracy_scores_nn = []

# モデルのトレーニングと評価を10回繰り返す
for i in range(10):
    # ニューラルネットワークモデルの初期化
    model_nn = MLPClassifier(random_state=42)


    # GridSearchCVの設定
    grid_search_nn = GridSearchCV(estimator=model_nn, param_grid=param_grid_nn, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# 訓練データでモデルをトレーニングし、最適なハイパーパラメータを取得
    grid_search_nn.fit(X_train_scaled_df_nn, y_train_nn)
    best_model_nn = grid_search_nn.best_estimator_

    
    # 訓練データでモデルをトレーニング
    #model_nn.fit(X_train_scaled_df_nn, y_train_nn)

    # 特徴量の重要度を取得し、リストに追加
    #importances_list_nn.append(best_model_nn.feature_importances_)

    # テストデータで予測確率を取得し、リストに追加
    y_scores_nn = best_model_nn.predict_proba(X_test_scaled_df_nn)[:, 1]
    predicted_probabilities_nn.append(y_scores_nn)

    # テストデータで予測を行う
    y_pred_nn = best_model_nn.predict(X_test_scaled_df_nn)
# Permutation Importanceの計算
    result_nn = permutation_importance(best_model_nn, X_test_scaled_df_nn, y_test_nn, n_repeats=10, random_state=42, n_jobs=-1)
    importances_list_nn.append(result_nn.importances_mean)

    # テストデータで予測確率を取得し、リストに追加
    y_scores_nn = best_model_nn.predict_proba(X_test_scaled_df_nn)[:, 1]
    predicted_probabilities_nn.append(y_scores_nn)

    # テストデータで予測を行う
    y_pred_nn = best_model_nn.predict(X_test_scaled_df_nn)
    
# 精度を計算してリストに追加
    accuracy_nn = accuracy_score(y_test_nn, y_pred_nn)
    accuracy_scores_nn.append(accuracy_nn)

# 10回の平均精度を計算
average_accuracy_nn = np.mean(accuracy_scores_nn)

# 平均精度を出力
print(f"\n平均精度 (10回): {average_accuracy_nn:.4f}")


# 特徴量の重要度の平均を計算
mean_importances_nn = np.mean(importances_list_nn, axis=0)

# 特徴量の名前と重要度の平均をデータフレームにまとめる
feature_importance_df_nn = pd.DataFrame({
    'Feature': X_train_nn.columns,
    'Importance': mean_importances_nn
})

# 重要度でソート
feature_importance_df_nn = feature_importance_df_nn.sort_values(by='Importance', ascending=False)
# 上位13項目を抽出
top_13_importance_df_nn = feature_importance_df_nn.head(13)

# グラフの作成
plt.figure(figsize=(10, 8))
plt.barh(top_13_importance_df_nn['Feature'], top_13_importance_df_nn['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.title('Top 13 Feature Importances (Random Forest)')
plt.gca().invert_yaxis()
plt.show()

# 平均予測確率を計算
mean_predicted_probabilities_nn = np.mean(predicted_probabilities_nn, axis=0)

from sklearn.metrics import confusion_matrix, roc_curve, auc


# テストデータで予測確率を取得
y_scores_nn = best_model_nn.predict_proba(X_test_scaled_df_nn)[:, 1]

# ROC計算用のしきい値を決定（スコアの一意な値を使用）
thresholds_nn = np.sort(np.unique(y_scores_nn))

# 感度（TPR）と特異度（1 - FPR）を計算
tpr_list_nn = []
specificity_list_nn = []

for threshold in thresholds_nn:
    # スコアをしきい値で2値化
    y_pred_nn = (y_scores_nn >= threshold).astype(int)
    
    # 混同行列を取得
    tn_nn, fp_nn, fn_nn, tp_nn = confusion_matrix(y_test_nn, y_pred_nn).ravel()
    
    # 感度と特異度を計算
    tpr_nn = tp_nn / (tp_nn+ fn_nn) if (tp_nn + fn_nn) > 0 else 0  # 感度
    specificity_nn = tn_nn / (tn_nn + fp_nn) if (tn_nn + fp_nn) > 0 else 0  # 特異度
    
    # リストに保存
    tpr_list_nn.append(tpr_nn)
    specificity_list_nn.append(specificity_nn)

# AUCの計算
roc_auc_nn = auc(np.array(specificity_list_nn), tpr_list_nn)

# ROC曲線を描画
plt.figure(figsize=(8, 6))
plt.plot( np.array(specificity_list_nn), tpr_list_nn, label=f"ROC Curve (AUC = {roc_auc_nn:.2f})", color="blue")

plt.plot([1, 0], [0, 1], linestyle="--", color="red", label="Random Guess")

# グラフの装飾
plt.xlim([1.0, 0.0])  # X軸を1から0に反転
plt.ylim([0.0, 1.0])
plt.xlabel("Specificity ")
plt.ylabel("Sensitivity")
plt.title("ROC Curve with TP, FP, TN, FN")
plt.legend(loc="lower left")
plt.grid()
plt.show()

from sklearn.metrics import confusion_matrix
# 混同行列の計算
tn_nn, fp_nn, fn_nn, tp_nn = confusion_matrix(y_test_nn, y_pred_nn).ravel()

# 感度と特異度の計算
sensitivity_nn = tp_nn / (tp_nn + fn_nn)  # 真陽性率
specificity_nn = tn_nn / (tn_nn + fp_nn)  # 真陰性率

print(f"Sensitivity: {sensitivity_nn:.4f}")
print(f"Specificity: {specificity_nn:.4f}")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 混同行列の計算
cm_nn = confusion_matrix(y_test_nn, y_pred_nn)

# 混同行列の表示
disp_nn = ConfusionMatrixDisplay(confusion_matrix=cm_nn, display_labels=["Negative", "Positive"])
disp_nn.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# 混同行列の要素を使って感度と特異度を計算
tn_nn, fp_nn, fn_nn, tp_nn = cm_nn.ravel()
sensitivity_nn = tp_nn / (tp_nn + fn_nn)  # 感度
specificity_nn = tn_nn / (tn_nn + fp_nn)  # 特異度

print(f"Sensitivity: {sensitivity_nn:.2f}")
print(f"Specificity: {specificity_nn:.2f}")